# Software Testing - Unittest
## Load python unittest module

In [3]:
import unittest

## Aspect Extraction
### Functions that need to be tested

In [ ]:
def get_vocabs(datasets):
    """Build vocabulary from an iterable of datasets objects

    Args:
        datasets: a list of dataset objects

    Returns:
        a set of all the words in the dataset

    """
    print("Building vocab...")
    vocab_words = set()
    vocab_tags = set()
    for dataset in datasets:
        for words, tags in dataset:
            #print(words)
            vocab_words.update(words)
            vocab_tags.update(tags)
    print("- done. {} tokens".format(len(vocab_words)))
    return vocab_words, vocab_tags


def get_char_vocab(dataset):
    """Build char vocabulary from an iterable of datasets objects

    Args:
        dataset: a iterator yielding tuples (sentence, tags)

    Returns:
        a set of all the characters in the dataset

    """
    vocab_char = set()
    for words, _ in dataset:
        for word in words:
            vocab_char.update(word)

    return vocab_char


def get_glove_vocab(filename):
    """Load vocab from file

    Args:
        filename: path to the glove vectors

    Returns:
        vocab: set() of strings
    """
    print("Building vocab Glove...")
    vocab = set()
    with open(filename) as f:
        for line in f:
            word = line.strip().split(' ')[0]
            vocab.add(word)
    print("- done. {} tokens".format(len(vocab)))
    return vocab


def write_vocab(vocab, filename):
    """Writes a vocab to a file

    Writes one word per line.

    Args:
        vocab: iterable that yields word
        filename: path to vocab file

    Returns:
        write a word per line

    """
    print("Writing vocab...")
    with open(filename, "w") as f:
        for i, word in enumerate(vocab):
            if i != len(vocab) - 1:
                f.write("{}\n".format(word))
            else:
                f.write(word)
    print("- done. {} tokens".format(len(vocab)))


def load_vocab(filename):
    """Loads vocab from a file

    Args:
        filename: (string) the format of the file must be one word per line.

    Returns:
        d: dict[word] = index

    """
    try:
        d = dict()
        with open(filename) as f:
            for idx, word in enumerate(f):
                word = word.strip()
                d[word] = idx

    except IOError:
        raise MyIOError(filename)
    return d


def export_trimmed_glove_vectors(vocab, glove_filename, trimmed_filename, dim):
    """Saves glove vectors in numpy array

    Args:
        vocab: dictionary vocab[word] = index
        glove_filename: a path to a glove file
        trimmed_filename: a path where to store a matrix in npy
        dim: (int) dimension of embeddings

    """
    embeddings = np.zeros([len(vocab), dim])
    with open(glove_filename) as f:
        for line in f:
            line = line.strip().split(' ')
            word = line[0]
            embedding = [float(x) for x in line[1:]]
            if word in vocab:
                word_idx = vocab[word]
                embeddings[word_idx] = np.asarray(embedding)

    np.savez_compressed(trimmed_filename, embeddings=embeddings)


def get_trimmed_glove_vectors(filename):
    """
    Args:
        filename: path to the npz file

    Returns:
        matrix of embeddings (np array)

    """
    try:
        with np.load(filename) as data:
            return data["embeddings"]

    except IOError:
        raise MyIOError(filename)


def get_processing_word(vocab_words=None, vocab_chars=None,
                    lowercase=False, chars=False, allow_unk=True):
    """Return lambda function that transform a word (string) into list,
    or tuple of (list, id) of int corresponding to the ids of the word and
    its corresponding characters.

    Args:
        vocab: dict[word] = idx

    Returns:
        f("cat") = ([12, 4, 32], 12345)
                 = (list of char ids, word id)

    """
    def f(word):
        # 0. get chars of words
        if vocab_chars is not None and chars == True:
            char_ids = []
            for char in word:
                # ignore chars out of vocabulary
                if char in vocab_chars:
                    char_ids += [vocab_chars[char]]

        # 1. preprocess word
        if lowercase:
            word = word.lower()
        if word.isdigit():
            word = NUM

        # 2. get id of word
        if vocab_words is not None:
            if word in vocab_words:
                word = vocab_words[word]
            else:
                if allow_unk:
                    word = vocab_words[UNK]
                else:
                    raise Exception("Unknow key is not allowed. Check that "\
                                    "your vocab (tags?) is correct")

        # 3. return tuple char ids, word id
        if vocab_chars is not None and chars == True:
            return char_ids, word
        else:
            return word

    return f


def _pad_sequences(sequences, pad_tok, max_length):
    """
    Args:
        sequences: a generator of list or tuple
        pad_tok: the char to pad with

    Returns:
        a list of list where each sublist has same length
    """
    sequence_padded, sequence_length = [], []

    for seq in sequences:
        seq = list(seq)
        seq_ = seq[:max_length] + [pad_tok]*max(max_length - len(seq), 0)
        sequence_padded +=  [seq_]
        sequence_length += [min(len(seq), max_length)]

    return sequence_padded, sequence_length


def pad_sequences(sequences, pad_tok, nlevels=1):
    """
    Args:
        sequences: a generator of list or tuple
        pad_tok: the char to pad with
        nlevels: "depth" of padding, for the case where we have characters ids

    Returns:
        a list of list where each sublist has same length

    """
    if nlevels == 1:
        max_length = max(map(lambda x : len(x), sequences))
        sequence_padded, sequence_length = _pad_sequences(sequences,
                                            pad_tok, max_length)

    elif nlevels == 2:
        max_length_word = max([max(map(lambda x: len(x), seq))
                               for seq in sequences])
        sequence_padded, sequence_length = [], []
        for seq in sequences:
            # all words are same length now
            sp, sl = _pad_sequences(seq, pad_tok, max_length_word)
            sequence_padded += [sp]
            sequence_length += [sl]

        max_length_sentence = max(map(lambda x : len(x), sequences))
        sequence_padded, _ = _pad_sequences(sequence_padded,
                [pad_tok]*max_length_word, max_length_sentence)
        sequence_length, _ = _pad_sequences(sequence_length, 0,
                max_length_sentence)
    if nlevels ==1:
        return sequence_padded, sequence_length, max_length
    else:
        return sequence_padded, sequence_length

def minibatches(data, minibatch_size):
    """
    Args:
        data: generator of (sentence, tags) tuples
        minibatch_size: (int)

    Yields:
        list of tuples

    """
    x_batch, y_batch = [], []
    for (x, y) in data:
        if len(x_batch) == minibatch_size:
            yield x_batch, y_batch
            x_batch, y_batch = [], []

        if type(x[0]) == tuple:
            x = zip(*x)
        x_batch += [x]
        y_batch += [y]

    if len(x_batch) != 0:
        yield x_batch, y_batch


def get_chunk_type(tok, idx_to_tag):
    """
    Args:
        tok: id of token, ex 4
        idx_to_tag: dictionary {4: "B-PER", ...}

    Returns:
        tuple: "B", "PER"

    """
    tag_name = idx_to_tag[tok]
    tag_class = tag_name.split('-')[0]
    tag_type = tag_name.split('-')[-1]
    return tag_class, tag_type


def get_chunks(seq, tags,message=None):
    """Given a sequence of tags, group entities and their position

    Args:
        seq: [4, 4, 0, 0, ...] sequence of labels
        tags: dict["O"] = 4

    Returns:
        list of (chunk_type, chunk_start, chunk_end)

    Example:
        seq = [4, 5, 0, 3]
        tags = {"B-PER": 4, "I-PER": 5, "B-LOC": 3}
        result = [("PER", 0, 2), ("LOC", 3, 4)]

    """
    default = tags[NONE]
    idx_to_tag = {idx: tag for tag, idx in tags.items()}
    chunks = []
    chunk_type, chunk_start = None, None
    for i, tok in enumerate(seq):
        # End of a chunk 1
        if tok == default and chunk_type is not None:
            # Add a chunk.
            chunk = (chunk_type, chunk_start, i)
            chunks.append(chunk)
            chunk_type, chunk_start = None, None

        # End of a chunk + start of a chunk!
        elif tok != default:
            tok_chunk_class, tok_chunk_type = get_chunk_type(tok, idx_to_tag)
            if chunk_type is None:
                chunk_type, chunk_start = tok_chunk_type, i
            elif tok_chunk_type != chunk_type or tok_chunk_class == "B":
                chunk = (chunk_type, chunk_start, i)
                chunks.append(chunk)
                chunk_type, chunk_start = tok_chunk_type, i
        else:
            pass

    # end condition
    if chunk_type is not None:
        chunk = (chunk_type, chunk_start, len(seq))
        chunks.append(chunk)
    #print(message + "{}{}{}".format(seq, tags, chunks))

    return chunks


## Aspect Aggregation
### Functions that need to be tested
The functions used for aspect aggregation are copied here because of some modification on them. The reason why we want to modify them is because some parts of the functions are not able to be tested (e.g. model prediction). Hence, copying functions to this file reduces the risk of having buggy code and also allows us to have more flexibilities.

In [115]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 13 02:23:50 2019

@author: Zhiwei and Friend(s)
"""

"""
###########################################
  _      _ _                    _          
 | |    (_| |                  (_)         
 | |     _| |__  _ __ __ _ _ __ _  ___ ___ 
 | |    | | '_ \| '__/ _` | '__| |/ _ / __|
 | |____| | |_) | | | (_| | |  | |  __\__ \
 |______|_|_.__/|_|  \__,_|_|  |_|\___|___/
                                           
###########################################
"""

import os
import re
import numpy as np
import pandas as pd
import itertools
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
import xml.etree.ElementTree as et

# NLTK
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Keras
from keras.preprocessing.text import text_to_word_sequence

"""
###############################################
  ______                _   _                 
 |  ____|              | | (_)                
 | |__ _   _ _ __   ___| |_ _  ___  _ __  ___ 
 |  __| | | | '_ \ / __| __| |/ _ \| '_ \/ __|
 | |  | |_| | | | | (__| |_| | (_) | | | \__ \
 |_|   \__,_|_| |_|\___|\__|_|\___/|_| |_|___/

###############################################
"""

# Flatten list of list
def _flatten(l):
    """
    This function will flatten a list of list to a list. (e.g. [[1],[2]] -> [1, 2])
    :arg {l} - a list of list
    :return - flattened list
    """
    return list(itertools.chain.from_iterable(l))

def _clean_text(text, stopwords=set(stopwords.words("english"))): #, lemmatizer=WordNetLemmatizer()):
    """
    This function is used for the preprocessing step, which will
    - convert text to lowercase
    - remove quotations surrounding the word (e.g. 'perks' -> perks)
    - handle some contraction of words (e.g. he's -> he is, can't -> cannot)
    - remove multiple consecutive spaces
    - remove the space that starts or ends in the sentence
    - remove stopwords
    (Note: the lemmatization has been done for this and we found that it did not provide a better result)
    :arg {text} - a string (sentence)
    :arg {stopwords} - a set of words 
    :return - preprocessed string
    """
    text = text.lower()
    text = re.sub(r"\'(\w*)\'", r"\1", text)
    text = re.sub(r"(he|she|it)\'s", r"\1 is", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    text = " ".join([w for w in word_tokenize(text) if not w in stopwords])
    return text

def _readXML(filename):
    """
    This function is to read SemEval Dataset in XML format. Here, we only 7 columns, which are:
    ['review', 'term', 'termPolarity', 'startIndex', 'endIndex','aspect', 'aspectPolarity']
    :arg {filename} - the dataset file (e.g. "Restaurant_Train.xml")
    :return - pandas dataframe
    """
    table = []
    row = [np.NaN] * 7
    
    for event, node in et.iterparse(filename, events=("start", "end")):

        if node.tag == "text":
            row[0] = node.text
        elif node.tag == "aspectTerms" and event == "start":
            row[1] = []
            row[2] = []
            row[3] = []
            row[4] = []
        elif node.tag == "aspectTerm" and event == "start":
            row[1].append(node.attrib.get("term").replace("-", " ").replace("/", " "))
            row[2].append(node.attrib.get("polarity"))
            row[3].append(int(node.attrib.get("from")))
            row[4].append(int(node.attrib.get("to")))
        elif node.tag == "aspectCategories" and event == "start":
            row[5] = []
            row[6] = []
        elif node.tag == "aspectCategory" and event == "start":
            row[5].append(node.attrib.get("category"))
            row[6].append(node.attrib.get("polarity"))
        elif node.tag == "aspectCategories" and event == "end":
            table.append(row)
            row = [np.NaN] * 7

    dfcols = ['review', 'term', 'termPolarity', 'startIndex', 'endIndex','aspect', 'aspectPolarity']
    data = pd.DataFrame(table, columns=dfcols)
    data["review"] = data["review"].str.replace("-", " ")
    data["review"] = data["review"].str.replace("/", " ")
    return data
    
def _add6PosFeautures(sentences, max_sent_len = 65):
    """
    This function is specially made for add 6 POS tag features for the model we have trained.
    :arg {sentences} - list of sentences
    :arg {max_sent_len} - the maximum sentence length (by default it would be 65)
    :return - pos features for given list of sentences
    """
    le = LabelEncoder()
    pos_tags = ["CC","NN","JJ","VB","RB","IN"]
    le.fit(pos_tags)
    input_data = np.zeros((len(sentences), max_sent_len, len(pos_tags)))
    
    for i, sentence in enumerate(sentences):
        words = text_to_word_sequence(sentence)
        tags = nltk.pos_tag(words)
        sentence_len = len(tags)
        
        for j in range(max_sent_len):
            if j< sentence_len :
                curr_tag = tags[j][1][:2] # only see the first two letters
                if curr_tag in pos_tags:                    
                    index = (le.transform([curr_tag]))[0]
                    input_data[i][j][index] = 1

    return np.asarray(input_data)

def _oneHotVectorize(df, mlb, le):
    """
    This function acts as a vectorizer that turns a list of aspects into one-hot vector.
    However, it is modified to accommodate a multilabel pattern.
    :arg {df} - a dataframe (in this case, it would be our dataset)
    :arg {mlb} - a multilabel binarizer (from module "sklearn")
    :arg {le} - a label encoder (from module "sklearn")
    :return - processed dataframe
    """
    df = df.apply(le.transform)
    df = mlb.fit_transform(df)
    return df

def _modified_performance_measure(preds, label=None):
    """
    This function is a modified version of _performance_measure.
    The original _performance_measure function needs a model as its first input.
    And the prediction by the model is unpredictable as it would be in a form of a list of 5 probabilities.
    Therefore, we modify the function by giving the prediction here and test whether the rest of the code is correct. 
    :arg {preds} - prediction in probabilities
    :arg {label} - the label for the input 
    :return - accuracy, precision, recall and f1 in a list
    """
    processed_preds = []
    for i in range(len(preds)):
        pred = list(map(lambda val: 1 if val > 0.175 else 0, preds[i]))
        processed_preds.append(pred)
        
    # return the prediction if no label is provided.
    # as this would be in the case where users just want to see the output of model given their inputs 
    if label is None:
        return processed_preds

    test_label = processed_preds
    true_label = label

    total_pos = .0
    total_neg = .0
    tp = .0 # True Positive
    tn = .0 # True Negative
    for i in range(len(test_label)):
        for j in range(len(test_label[0])):
            if test_label[i][j] == 1:
                total_pos += 1
                if true_label[i][j] ==1:
                    tp +=1
            if test_label[i][j] == 0:
                total_neg += 1
                if true_label[i][j] ==0:
                    tn += 1
    fp = total_neg - tn # False Positive
    fn = total_pos - tp # False Negative
    precision = tp/(tp + fp)
    recall = tp/total_pos
    f1 = 2 * (precision * recall)/(precision + recall)
    acc = (tp + tn)/(total_pos + total_neg)

    return list(map(lambda x:round(x,2),[acc, precision, recall, f1]))

In [8]:
class TestFunc(unittest.TestCase):

    def test_flatten(self):
        """
        Test function _flatten
        """
        self.assertEqual(_flatten([[1], [2]]), [1, 2])
        self.assertEqual(_flatten([["a","b"], ["c"]]), ["a", "b", "c"])
        self.assertFalse(_flatten([[[1]], [[2]]]) == [[1], 2]) # Output should be [[1], [2]]
        self.assertTrue(_flatten([[1], ["a"], [2], ["b"]]), [1, "a", 2, "b"])
        self.assertTrue(_flatten([[]]) == [])
        
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [9]:
class TestFunc(unittest.TestCase):

    def test_clean_text(self):
        """
        Test function _clean_text
        """
        # test for lowercase
        self.assertEqual(_clean_text("ABC"), "abc")
        # test for stopword removal
        self.assertEqual(_clean_text("the food is nice."), "food nice .")
        # test for removing quotations that surround words
        self.assertEqual(_clean_text("'food' 'is' 'nice'"), "food nice")
        # test for contraction he's, she's and it's (if not expanded, then "'s" will not be treated as a stopword)
        self.assertEqual(_clean_text("He's a good boy and she's a good girl. It's not a good dog."), \
                                     "good boy good girl . good dog .")
        # test for contraction can't, 'll, n't (if not expanded, then they might not be treated as a stopword)
        self.assertEqual(_clean_text("I don't think I can't win the game but I'll lose him if you didn't ask me."), \
                                     "think win game lose ask .")
        # test for contraction i'm and 're (if not expanded, then "'s" will not be treated as a stopword)
        self.assertEqual(_clean_text("I'm not going to that place but you're going to that place."), \
                                     "going place going place .")
        # test for multiple consecutive whitespace removal
        self.assertEqual(_clean_text("dim     sum     is     good"), \
                                     "dim sum good")
        # test for starting and ending whitespace removal
        self.assertEqual(_clean_text(" dim sum is good "), \
                                     "dim sum good")

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.033s

OK


In [29]:
class TestFunc(unittest.TestCase):

    def setUp(self):
        """
        This function is to create some examples used for testing
        """
        self.test_xml_file1 = "./Datasets/test_readXML_1.xml"
        self.test_xml_file2 = "./Datasets/test_readXML_2.xml"

    def test_readXML(self):
        """
        test function _readXML
        """
        # Dataframes created from our function
        df1 = _readXML(self.test_xml_file1)
        df2 = _readXML(self.test_xml_file2)

        # Manually created dataframes
        check_df1 = pd.DataFrame(columns=['review', 'term', 'termPolarity', 'startIndex', 'endIndex','aspect', 'aspectPolarity'])
        check_df1 = check_df1.append(pd.Series(['The food is nice!', ['food'], \
                                    ['positive'], [4], [8], ['food'], ['positive']], \
                                   index=check_df1.columns ), ignore_index=True)
        check_df1 = check_df1.append(pd.Series(['It would be better if there are some peppers on it.', ['peppers'], \
                                    ['neutral'], [37], [44], ['food'], ['neutral']], \
                                   index=check_df1.columns ), ignore_index=True)
        
        check_df2 = pd.DataFrame(columns=['review', 'term', 'termPolarity', 'startIndex', 'endIndex','aspect', 'aspectPolarity'])
        check_df2 = check_df2.append(pd.Series(['I hate that waiter.', ['waiter'], \
                                    ['negative'], [12], [18], ['service'], ['negative']], \
                                   index=check_df2.columns ), ignore_index=True)
        check_df2 = check_df2.append(pd.Series(['The fish and chips is so delicious.', ['fish and chips'], \
                                    ['positive'], [4], [18], ['food'], ['positive']], \
                                   index=check_df2.columns ), ignore_index=True)        

        # Check whether they are equal
        self.assertTrue(df1.equals(check_df1))
        self.assertTrue(df2.equals(check_df2))        
        
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.032s

OK


In [76]:
import unittest

class TestFunc(unittest.TestCase):

    def setUp(self):
        """
        This function is to create some examples used for testing
        """
        self.sentences1 = [
            "The food is nice!",
            "It would be better if there are some peppers on it."
        ]
        self.sentences2 = [
            "I hate that waiter.",
            "The fish and chips is so delicious."
        ]

    def test_add6PosFeatures(self):
        """
        test function _add6PosFeatures
        """ 
        # Part-of-Speech sequence = ["CC","IN","JJ","NN","RB","VB"]
        # Max Sentence Length = 65
        # The Part-of-Speech features provided by our function
        pos_features1 = _add6PosFeautures(self.sentences1)
        pos_features2 = _add6PosFeautures(self.sentences2)
        
        # Manually created Part-of-Speech features
        features1 = np.zeros((len(self.sentences1), 65, 6))
        for i, sentence in enumerate(self.sentences1):
            words1 = text_to_word_sequence(sentence)
            pos1 = nltk.pos_tag(words1)
            for j, pos in enumerate(pos1):
                if pos[1][:2] == "CC":
                    features1[i][j][0] = 1
                elif pos[1][:2] == "NN":
                    features1[i][j][3] = 1
                elif pos[1][:2] == "JJ":
                    features1[i][j][2] = 1
                elif pos[1][:2] == "VB":
                    features1[i][j][5] = 1
                elif pos[1][:2] == "RB":
                    features1[i][j][4] = 1
                elif pos[1][:2] == "IN":
                    features1[i][j][1] = 1
                    
        features2 = np.zeros((len(self.sentences2), 65, 6))
        for i, sentence in enumerate(self.sentences2):
            words2 = text_to_word_sequence(sentence)
            pos2 = nltk.pos_tag(words2)
            for j, pos in enumerate(pos2):
                if pos[1][:2] == "CC":
                    features2[i][j][0] = 1
                elif pos[1][:2] == "NN":
                    features2[i][j][3] = 1
                elif pos[1][:2] == "JJ":
                    features2[i][j][2] = 1
                elif pos[1][:2] == "VB":
                    features2[i][j][5] = 1
                elif pos[1][:2] == "RB":
                    features2[i][j][4] = 1
                elif pos[1][:2] == "IN":
                    features2[i][j][1] = 1
        
        # Check whether they are equal
        self.assertTrue(np.array_equal(pos_features1, features1))
        self.assertTrue(np.array_equal(pos_features2, features2))

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.017s

OK


In [105]:
class TestFunc(unittest.TestCase):

    def setUp(self):
        """
        This function is to create some examples used for testing
        """
        self.unique_asp = ["service","food","price","ambience","anecdotes/miscellaneous"]
        self.mlb = MultiLabelBinarizer(classes=[i for i in range(5)])
        self.le = LabelEncoder()
        self.le.fit(self.unique_asp)

    def test_oneHotVectorize(self):
        """
        test function _oneHotVectorize
        """
        df1 = pd.DataFrame([
            [["service"]],
            [["service","price"]]
          ], columns=["aspect"])
        
        df2 = pd.DataFrame([
            [["food"]],
            [["food","service","price"]]
          ], columns=["aspect"])
        
        # encoded labels by our function
        labels1 = _oneHotVectorize(df1["aspect"], self.mlb, self.le)
        labels2 = _oneHotVectorize(df2["aspect"], self.mlb, self.le)        
        
        # manually encoded labels  
        check_df1 = []
        check_df2 = []
        for i in range(2):
            l = []
            l2 = []
            for j in range(5):
                l.append(0)
                l2.append(0)
            check_df1.append(l)
            check_df2.append(l2)
            
        for i,row in enumerate(df1["aspect"]):
            for j,aspect in enumerate(row):
                if aspect == "service":
                    check_df1[i][4]=1
                elif aspect == "food":
                    check_df1[i][2]=1
                elif aspect == "price":
                    check_df1[i][3]=1
                elif aspect == "ambience":
                    check_df1[i][0]=1
                else:
                    check_df1[i][1]=1
                    
        for i,row in enumerate(df2["aspect"]):
            for j,aspect in enumerate(row):
                if aspect == "service":
                    check_df2[i][4]=1
                elif aspect == "food":
                    check_df2[i][2]=1
                elif aspect == "price":
                    check_df2[i][3]=1
                elif aspect == "ambience":
                    check_df2[i][0]=1
                else:
                    check_df2[i][1]=1
        check_df1 = np.asarray(check_df1)
        check_df2 = np.asarray(check_df2)
        
        # check whether they are equal
        self.assertTrue(np.array_equal(labels1, check_df1))
        self.assertTrue(np.array_equal(labels2, check_df2))
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


In [129]:
import unittest

class TestFunc(unittest.TestCase):

    def setUp(self):
        """
        This function is to create some examples used for testing
        """
        self.predicted1 = [
            [0.0,0.0,0.45,0.45,0.1],
            [0.0,0.0,0.9,0.05,0.05],
            [1.0,0.0,0.0,0.0,0.0],
            [0.0,0.0,0.0,0.9,0.1]
        ]

        self.actual1 = [
            [0,0,1,0,1],
            [0,0,1,0,0],
            [0,0,0,1,0],
            [1,1,1,1,0]
        ]
        
        self.predicted2 = [
            [1.0,0.0,0.0,0.0,0.0],
            [0.0,0.0,0.0,1.0,0.0]
        ]

        self.actual2 = [
            [0,0,0,1,0],
            [1,1,1,1,0]
        ]

    def test_modified_performance_measure(self):
        # result from our function
        result1 = _modified_performance_measure(self.predicted1, self.actual1)
        result2 = _modified_performance_measure(self.predicted2, self.actual2)
        
        # real result
        real_result1 =[0.65, 0.38, 0.6, 0.46]
        real_result2 = [0.50, 0.20, 0.50, 0.29]
        
        # check whether they are equal
        self.assertTrue(result1 == real_result1)
        self.assertTrue(result2 == real_result2)

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
